# Basic Decision Trees
## The Problem
Let's start with a concrete example of the type of problem we are going to discuss.  Supposed that you are a bird watcher and you have been tracking if your favorite Barn Swallow shows up during lunch time.  You have been keeping track of a number of different variables (in machine learning we call these "features") every morning along with whether or not your friend shows up during lunch.  The features you record are:

| Feature Name | Possible Values      |
|--------------|----------------------|
| Outlook      |Sunny, Overcast, Rain |
| Temperature  |Cool, Mild, Hot       |
| Humidity     |High, Normal          |
| Windy        |True, False           |

This is called "categorical data" because each feature takes on a single value from a finite set of possible values.  Categorical data is often created by grouping data that is otherwise continuous (e.g. Temperature gets grouped into Cool, Mild or Hot based on some criterion).

The goal of this type of machine learning problem is to determine, based on a particular measurement or "instance" of the features values (i.e. Outlook, Temperature, Humidity, Windy), whether our bird friend will show up.  This is called a binary classification problem because we are classifying instances (e.g. Outlook=Overcast, Temperature=Hot, Humidity=High, Windy=True) as either thumbs up (the bird shows up) or thumbs down (the bird does not show up).

## A Solution
The mathematical view of the above problem is that we are looking for a function.  Functions map each input to one and only one output.  The function in this case maps (Outlook, Temperature, Humidity, Windy) to thumbs up or thumbs down.  How many different possible functions are there to choose from?

There are $3 \cdot 3 \cdot 2 \cdot 2 = 36$ possible inputs (Outlook has 3 possibilities, Temperature has 3, Humidity has 2 and Windy has 2 possibilities).  For each input the function can evaluate to one of two values (thumbs up or thumbs down).  Thus there are $2^{36}=68,719,476,736$ possible functions associated with this simple problem.

### A Little Theory
Does it surprise you that a problem this simple has so many different possible solutions?  This is just a fact of life for machine learning practitioners.  Consider the case of a problem with $N$ features each of which can take on 2 values in a binary classification problem.  In this case, the number of possible solutions/functions is

$$2^{2^{N}}$$

(each of the $2^{N}$ possible inputs has 2 possible outputs).  Note that the size of the solution space grows like a double exponential with the number of features involved.  Clearly a brute force search of the solution space is not viable.

It is useful to note that even just to explicitly represent the single "correct" function that we are after requires exponential memory - we need to remember the output for each of the $2^{N}$ possible inputs.  Of course if we know the correct output for every input we don't have a learning problem.  We have (in practice at least) a compression problem.  If we choose to use lossy compression then we are effectively approximating a known function.  Neither of these is learning.

### What is Learning?
There are a number of different types of learning problems.  For the most part they all involve trying to approximate a function based on instance data (sets of features) which may (supervised learning) or may not (unsupervised learning) have "correct" outputs associated with them.

Typically the amount of data available to learn from is a tiny fraction of the set of all possible inputs (i.e. all possible combinations of feature values).  Given this, it is reasonable ask if "learning" is even possible.  How can we possibly know what value a function will take on without observing what it actually evaluates to?  The answers is that it is not possible to learn "pathological" functions but it is possible to approximate functions that exhibit "large scale" patterns in their output which can be detected from sparse sampling.  We won't attempt here to rigorously defined the conditions under which "generalization" is possible.  We will simply state that if a data set of interest contains patterns that generalize to the full domain it is possible (at least in theory) to learn and exploit them.

### A Function?  Really?
It is quite reasonable to question whether a function is really what we are after here.  Surely whether or not our friendly Barn Swallow shows up depends on more than just some crude measures of the weather!  This is most likely true and is often true in real world cases as well.  There may be "unobserved" features such as the proximity of cats at lunch time or numerous other factors related to Barn Swallow psychology.  The best we can really hope for is to approximate the "real" underlying function using just the data that is available.  There will be "modeling" error that can't be avoided.  This is not quite as bad as it sounds since approximation is probably the best we can hope for any way given the sparseness of the data typically available.  We will come back to this point later when we discuss bias and variance.

### How Do We Approximate the Function?
Before we tackle the question of how to approximate the function we are after it is useful to consider if it possible to always represent the function exactly if we have a complete set of inputs and outputs.  In machine learning choosing the manner in which the function will be represented is called choosing the "hypothesis space."  The specific function that is choosen based on the data is called the "hypothesis."   The hypothesis is restricted to be a member (in a set theoretic sense) of the hypothesis space.  In general the choice of hypothesis space can dramaticaly impact the quality of the resulting hypothesis (i.e. how well we are able to approximate the function we are after) and the amount of work necessary to pick the best hypothesis.  In the case at hand - supervised learning of binary classification over categorical data - we are in luck since a simple hypothesis space is both fairly obvious and capable of exactly representing the function we are after.

A simple way to represent a function that captures the observed instances is simply a series of if-then statements:

```python
if   outlook=='Sunny' and temperature=='Hot' and humidity=='High' and windy==False: return 'thumbs down'
elif outlook=='Sunny' and temperature=='Hot' and humidity=='High' and windy==True:  return 'thumbs down'
#...
```

It should be fairly obvious that this hypothesis space (a series of if-then statements over all the features) is capable of caturing the function we are after exactly **if** we have the complete set of inputs and outputs.  The problem, as we mentioned before, however is that this is not learning.  If we already have all the inputs and outputs there is nothing to learn!  If we just use this "series of rules" approach on sparse input we are left with no ability to generalize to inputs that have not been observed.  The if-then approach is promising however.  Consider the following instance data from which we wish to learn an approximate classifier function:

In [15]:
import pandas as pd
import itable
from IPython.display import display
values = zip(['Outlook', 'Temperature', 'Humidity', 'Windy', 'Barn Swallow?'],
    ['Sunny', 'Hot', 'High', False, 'thumbs down'], 
    ['Sunny', 'Hot', 'High', True, 'thumbs down'], 
    ['Overcast', 'Hot', 'High', False, 'thumbs up'],
    ['Rain', 'Mild', 'High', False, 'thumbs up'],
    ['Rain', 'Cool', 'Normal', False, 'thumbs up'],
    ['Rain', 'Cool', 'Normal', True, 'thumbs down'],
    ['Overcast', 'Cool', 'Normal', True, 'thumbs up'],
    ['Sunny', 'Mild', 'High', False, 'thumbs down'], 
    ['Sunny', 'Cool', 'Normal', False, 'thumbs up'],
    ['Rain', 'Mild', 'Normal', False, 'thumbs up'],
    ['Sunny', 'Mild', 'Normal', True, 'thumbs up'],
    ['Overcast', 'Mild', 'High', True, 'thumbs up'],
    ['Overcast', 'Hot', 'Normal', False, 'thumbs up'],
    ['Rain', 'Mild', 'High', True, 'thumbs down'])
df = pd.DataFrame({row[0]: row[1:] for row in values})
df = df.ix[:,['Outlook', 'Temperature', 'Humidity', 'Windy', 'Barn Swallow?']]
display(itable.PrettyTable(df, tstyle=itable.TableStyle(theme="theme1"), center=True))

Outlook,Temperature,Humidity,Windy,Barn Swallow?
Sunny,Hot,High,False,thumbs down
Sunny,Hot,High,True,thumbs down
Overcast,Hot,High,False,thumbs up
Rain,Mild,High,False,thumbs up
Rain,Cool,Normal,False,thumbs up
Rain,Cool,Normal,True,thumbs down
Overcast,Cool,Normal,True,thumbs up
Sunny,Mild,High,False,thumbs down
Sunny,Cool,Normal,False,thumbs up
Rain,Mild,Normal,False,thumbs up
